In [1]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [37]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('train.csv', 'train')# Added payments.csv for specific handling
]

# Connect to the MySQL database
import pymysql

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='renurana',
    database='new_media'
)
cursor = conn.cursor()

cursor = conn.cursor()

# Folder containing the CSV files
folder_path = r"C:\Users\Admin\OneDrive\Documents\social media folder"

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing train.csv
NaN values before replacement:
User_ID                       0
Age                           0
Gender                        1
Platform                      1
Daily_Usage_Time (minutes)    1
Posts_Per_Day                 1
Likes_Received_Per_Day        1
Comments_Received_Per_Day     1
Messages_Sent_Per_Day         1
Dominant_Emotion              1
dtype: int64



OperationalError: (1054, "Unknown column 'Daily_Usage_Time_(minutes)' in 'field list'")

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
import numpy as np

In [10]:
db = pymysql.connect(
    host='localhost',
    user='root',
    password='renurana',
    database='new_media'
)
cur=db.cursor()

In [13]:
query="""alter table train
rename column `Daily_Usage_Time_(minutes)`  to `daily_usage_time_minutes`;"""

cur.execute(query)
data=cur.fetchall()

data

OperationalError: (1054, "Unknown column 'Daily_Usage_Time_(minutes)' in 'train'")

# What are the daily usage times for all users?

In [14]:
query="""SELECT daily_usage_time_minutes,User_ID
from train
limit 10; """

cur.execute(query)
data=cur.fetchall()

data

((120.0, '1'),
 (90.0, '2'),
 (60.0, '3'),
 (200.0, '4'),
 (45.0, '5'),
 (150.0, '6'),
 (85.0, '7'),
 (110.0, '8'),
 (55.0, '9'),
 (170.0, '10'))

# How many users are on each platform?

In [15]:
query="""select count(User_ID), Platform
from train
group by train.Platform """

cur.execute(query)
data=cur.fetchall()

df=pd.DataFrame(data,columns=["no. of users","Platform"])
df

,no. of users,Platform
0,250,Instagram
1,200,Twitter
2,190,Facebook
3,120,LinkedIn
4,80,Whatsapp
5,80,Telegram
6,80,Snapchat
7,1,None


# Who are the male users?


In [32]:
query="""select User_ID,Age,Platform 
from train 
where Gender='Male' 
"""

cur.execute(query)
data=cur.fetchall()

data

(('2', '30', 'Twitter'),
 ('5', '33', 'LinkedIn'),
 ('6', '21', 'Instagram'),
 ('10', '31', 'Instagram'),
 ('13', '34', 'LinkedIn'),
 ('15', '28', 'Twitter'),
 ('18', '27', 'Twitter'),
 ('21', '26', 'Facebook'),
 ('23', '29', 'LinkedIn'),
 ('26', '31', 'Facebook'),
 ('29', '30', 'Twitter'),
 ('32', '28', 'Instagram'),
 ('35', '29', 'Facebook'),
 ('38', '27', 'Twitter'),
 ('41', '31', 'LinkedIn'),
 ('43', '29', 'Twitter'),
 ('46', '30', 'Instagram'),
 ('49', '32', 'Instagram'),
 ('51', '27', 'Instagram'),
 ('55', '31', 'LinkedIn'),
 ('57', '25', 'Telegram'),
 ('60', '27', 'Facebook'),
 ('62', '23', 'Whatsapp'),
 ('65', '25', 'Twitter'),
 ('67', '28', 'Instagram'),
 ('69', '27', 'Telegram'),
 ('72', '35', 'Twitter'),
 ('75', '25', 'Telegram'),
 ('78', '26', 'Twitter'),
 ('81', '31', 'Telegram'),
 ('84', '22', 'Twitter'),
 ('87', '31', 'Telegram'),
 ('90', '30', 'Twitter'),
 ('93', '24', 'Telegram'),
 ('95', '26', 'Instagram'),
 ('98', '35', 'Whatsapp'),
 ('102', '30', 'Twitter'),
 ('105'

# Who are the users, sorted by age?

In [36]:
query="""select User_ID,Age,Platform
from train
order by (train.Age )
"""

cur.execute(query)
data=cur.fetchall()

data

(('Tabii', ' işte mevcut veri kümesini 1000 satıra tamamlıyorum:', None),
 ('888', '21', 'Snapchat'),
 ('934', '21', 'Instagram'),
 ('873', '21', 'Facebook'),
 ('588', '21', 'Snapchat'),
 ('973', '21', 'Facebook'),
 ('6', '21', 'Instagram'),
 ('316', '21', 'Facebook'),
 ('247', '21', 'Twitter'),
 ('988', '21', 'Snapchat'),
 ('88', '21', 'Snapchat'),
 ('647', '21', 'Twitter'),
 ('616', '21', 'Facebook'),
 ('916', '21', 'Facebook'),
 ('447', '21', 'Twitter'),
 ('747', '21', 'Twitter'),
 ('16', '21', 'Facebook'),
 ('847', '21', 'Twitter'),
 ('188', '21', 'Snapchat'),
 ('734', '21', 'Instagram'),
 ('488', '21', 'Snapchat'),
 ('673', '21', 'Facebook'),
 ('547', '21', 'Twitter'),
 ('234', '21', 'Instagram'),
 ('116', '21', 'Facebook'),
 ('406', '21', 'Instagram'),
 ('373', '21', 'Facebook'),
 ('688', '21', 'Snapchat'),
 ('606', '21', 'Instagram'),
 ('334', '21', 'Instagram'),
 ('573', '21', 'Facebook'),
 ('534', '21', 'Instagram'),
 ('147', '21', 'Twitter'),
 ('34', '21', 'Instagram'),
 ('90

# -- What is the total number of messages sent by users aged 25 or younger?


In [37]:
query="""select sum(Messages_Sent_Per_Day)
from train
where Age<=25
"""

cur.execute(query)
data=cur.fetchall()

data

((8380.0,),)

# Which users have a daily usage time greater than 100 minutes?

In [40]:
query="""select User_ID,daily_usage_time_minutes
from train
where daily_usage_time_minutes>100
"""

cur.execute(query)
data=cur.fetchall()

df=pd.DataFrame(data,columns=["user id","daily usage"])
df

,user id,daily usage
0,1,120.0
1,4,200.0
2,6,150.0
3,8,110.0
4,10,170.0
...,...,...
335,989,170.0
336,992,105.0
337,995,150.0
338,998,110.0


# What is the average daily usage time by platform?

In [18]:
query="""select round(avg(daily_usage_time_minutes),2),Platform
from train
group by Platform
"""

cur.execute(query)
data=cur.fetchall()

df=pd.DataFrame(data,columns=["daily usage","Platform"])
df

,daily usage,Platform
0,153.40,Instagram
1,83.75,Twitter
2,72.11,Facebook
3,55.83,LinkedIn
4,87.50,Whatsapp
5,78.12,Telegram
6,90.00,Snapchat
7,NaN,None


# Which users post more than 5 times a day and receive more than 50 likes per day?

In [27]:
query="""SELECT 
    User_ID, Gender, Likes_Received_Per_Day, Posts_Per_Day
FROM
    train
WHERE
    Posts_Per_Day >= 5
        AND Likes_Received_Per_Day >= 50;
"""

cur.execute(query)
data=cur.fetchall()

df=pd.DataFrame(data,columns=["user id","gender","likes per day","posts per day"])
df

,user id,gender,likes per day,posts per day
0,4,Female,100.0,8.0
1,10,Male,80.0,5.0
2,14,Female,90.0,7.0
3,20,Female,70.0,5.0
4,24,Non-binary,95.0,8.0
...,...,...,...,...
221,967,Male,78.0,5.0
222,977,Female,75.0,6.0
223,983,Female,60.0,5.0
224,989,Female,83.0,6.0


# What is the distribution of dominant emotions among users?

In [15]:
query="""SELECT count(User_ID), Dominant_Emotion
FROM train
group by Dominant_Emotion
"""

cur.execute(query)
data=cur.fetchall()

df=pd.DataFrame(data,columns=["user id","emotions"])
df

,user id,emotions
0,200,Happiness
1,130,Anger
2,200,Neutral
3,170,Anxiety
4,140,Boredom
5,160,Sadness
6,1,None


# Who are the top 5 users with the highest daily usage times?

In [16]:
query="""SELECT User_ID, daily_usage_time_minutes
FROM train
order by daily_usage_time_minutes desc
limit 5
"""

cur.execute(query)
data=cur.fetchall()

df=pd.DataFrame(data,columns=["user id","daily usage time"])
df

,user id,daily usage time
0,404,200.0
1,4,200.0
2,904,200.0
3,504,200.0
4,104,200.0


#  Which platform has the highest average number of likes received per day?

In [20]:
query="""SELECT Platform, avg(Likes_Received_Per_Day)
FROM train
group by Platform
order by avg(Likes_Received_Per_Day) desc
"""

cur.execute(query)
data=cur.fetchall()

df=pd.DataFrame(data,columns=["user id","Likes_Received_Per_Day"])
df

,user id,Likes_Received_Per_Day
0,Instagram,79.272000
1,Whatsapp,36.450000
2,Twitter,35.245000
3,Snapchat,30.450000
4,Telegram,29.825000
5,Facebook,19.726316
6,LinkedIn,12.875000
7,None,NaN


# What is the gender distribution across different social media platforms?

In [26]:
query="""SELECT Gender, Platform,sum(User_ID)
FROM train
group by Gender,Platform
order by Platform
"""

cur.execute(query)
data=cur.fetchall()

df=pd.DataFrame(data,columns=["gender","platform","user id",])
df

,gender,platform,user id
0,None,None,0.0
1,21,Facebook,1046.0
2,Non-binary,Facebook,64268.0
3,Male,Facebook,18400.0
4,27,Facebook,1020.0
5,Female,Facebook,5470.0
6,29,Facebook,3108.0
7,33,Facebook,1058.0
8,Female,Instagram,73518.0
9,Male,Instagram,37524.0


# Which users express "Happiness" and use Instagram for more than 150 minutes per day?

In [29]:
query="""SELECT User_ID, Dominant_Emotion, Platform,daily_usage_time_minutes
FROM train
where Dominant_emotion='Happiness'  and  Platform='Instagram' and daily_usage_time_minutes>150
order by daily_usage_time_minutes asc
"""

cur.execute(query)
data=cur.fetchall()

df=pd.DataFrame(data,columns=["user","emotion","platform","daily_usage_time_minutes"])
df

,user,emotion,platform,daily_usage_time_minutes
0,77,Happiness,Instagram,155.0
1,177,Happiness,Instagram,155.0
2,277,Happiness,Instagram,155.0
3,377,Happiness,Instagram,155.0
4,477,Happiness,Instagram,155.0
...,...,...,...,...
105,524,Happiness,Instagram,190.0
106,624,Happiness,Instagram,190.0
107,724,Happiness,Instagram,190.0
108,824,Happiness,Instagram,190.0


# Who are the top 10 users based on their daily interaction (likes + comments + messages)?

In [36]:
query="""SELECT User_ID, (Likes_Received_Per_Day+Comments_Received_Per_Day+Messages_Sent_Per_Day)
FROM train
order by User_ID desc
limit 10
"""

cur.execute(query)
data=cur.fetchall()

df=pd.DataFrame(data,columns=["User id","likes+comments+messages"])
df

,User id,likes+comments+messages
0,Tabii,NaN
1,999,44.0
2,998,100.0
3,997,30.0
4,996,71.0
5,995,125.0
6,994,61.0
7,993,75.0
8,992,105.0
9,991,24.0
